# Coursera Capstone Project

# 1.	Background & Problem Statement

I reside in the Charlotte, North Carolina. After living here for 10 years, I moved to Pittsburgh, Pennsylvania for 5 years and have recently moved back to Charlotte about 2 years back.

Charlotte was always a vibrant metropolitan area supported by a family oriented suburban cities. Having come back after 5 years I see that the city has growth exponentially with many families preferring to move to warmer climate with good economic prospects. Charlotte fits this description. 

The growth boom is wonderful to watch. While looking for a new home upon relocating back to Charlotte, we looked at the various suburban cities in the Charlotte Metropolitan Area for a home and we found that while you can buy a house in any nearby city, the amenities were not always the best in the cities with newer neighborhoods.

So my goal is to analyze which cities surrounding the Charlotte Metropolitan and solve for 2 problem from 2 different perspective:
1.	Home Seekers: Assist people in evaluating various surrounding cities to enable them to find the location which has the amenities that they care about
2.	Business Owners: Assist businesses in evaluating under served market to analyze what business opportunity may exist in a particular location.

# Cluster #2 Analysis:
In this section, we study the attributes of second cluster. Cluster #2 was the city of Davidson itself which was more blue collar based on the venues in it.

--- The following sections brings in the required libraries to process and analyze the dataset.

In [2]:
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
print('Package Installed!')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    openssl-1.1.1f             |       h516909a_0         2.1 MB  conda-forge
    certifi-2019.11.28         |   py36h9f0ad1d_1         149 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1

In [3]:
#import pandas as pd # library for data analsysis
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
print('Panda Import Complete!')

Panda Import Complete!


In [4]:
import numpy as np # library to handle data in a vectorized manner
import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
print('Component Import Complete!')

Component Import Complete!


In [5]:
!conda install -c conda-forge/label/cf202003 folium
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
print('All Libraries imported!')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |   py36h9f0ad1d_1         149 KB  conda-forge/label/cf202003
    branca-0.4.0               |             py_0          26 KB  conda-forge/label/cf202003
    vincent-0.4.4              |             py_1          28 KB  conda-forge/label/cf202003
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge/label/cf202003
    folium-0.10.1              |             py_0          59 KB  conda-forge/label/cf202003
    openssl-1.1.1e             |       h516909a_0         2.1 MB  conda-forge/label/cf202003
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages wi

In this section, we study the attributes of Charlotte, North Carolina

In [7]:
Loc_Value = 'Charlotte, NC'

geolocator = Nominatim(user_agent="CLT")
location = geolocator.geocode(Loc_Value)
CLT_LAT = location.latitude
CLT_LON = location.longitude
print('The geograpical coordinate of Charlotte, NC are {}, {}.'.format(CLT_LAT, CLT_LON))

The geograpical coordinate of Charlotte, NC are 35.2270869, -80.8431268.


In this section, we import the data of cties in North Carolina.

In [8]:
DataURL = 'https://raw.githubusercontent.com/amitarya76/Coursera_Capstone/master/NC_CITY_DATA.csv'
Charlotte_df = pd.read_csv(DataURL)
Charlotte_df.head()

,city,city_ascii,state_id,state_name,county_fips,county_name,county_fips_all,county_name_all,lat,lng,population,density,source,military,incorporated,timezone,ranking,zips,id
0,Trenton,Trenton,NC,North Carolina,37103,Jones,37103,Jones,35.0640,-77.3553,285,484,polygon,False,True,America/New_York,3,28585,1840016445
1,Stanfield,Stanfield,NC,North Carolina,37167,Stanly,37167,Stanly,35.2336,-80.4303,1517,131,polygon,False,True,America/New_York,3,28163,1840016365
2,Sanford,Sanford,NC,North Carolina,37105,Lee,37105,Lee,35.4874,-79.1772,34557,389,polygon,False,True,America/New_York,3,27330 27332 27331,1840015421
3,Cofield,Cofield,NC,North Carolina,37091,Hertford,37091,Hertford,36.3567,-76.9105,382,46,polygon,False,True,America/New_York,3,27922,1840017704
4,Thomasville,Thomasville,NC,North Carolina,37057,Davidson,37057,Davidson,35.8813,-80.0807,26615,612,polygon,False,True,America/New_York,3,27360 27361,1840015361


In this section, we initiate the FourSquare API to bring in location data.

In [9]:
CLIENT_ID = 'VMGZKNE4PKH1F3KSSMMMPYQQV10FLEYBQRDGUV4NX5BDUVX1'
CLIENT_SECRET = 'LL0HD4WKVAA4C5YUO0AAMLM251D2JEVPRLVCLPW54GU21VRB'
VERSION = '20200331'
LIMIT = 100
RADIUS = 5000
CLUSTERS = 3

In this section, we filter for the cities in the Mecklenburg county. These are the cities in the Charlotte Metropolitan Area that we will study.

In [10]:
Charlotte_NC = Charlotte_df[Charlotte_df['county_name'].str.contains('Mecklenburg')].reset_index(drop = True)
Charlotte_NC

,city,city_ascii,state_id,state_name,county_fips,county_name,county_fips_all,county_name_all,lat,lng,population,density,source,military,incorporated,timezone,ranking,zips,id
0,Pineville,Pineville,NC,North Carolina,37119,Mecklenburg,37119,Mecklenburg,35.0864,-80.8915,8746,508,polygon,False,True,America/New_York,3,28134,1840016354
1,Davidson,Davidson,NC,North Carolina,37119,Mecklenburg,37119,Mecklenburg,35.4862,-80.8271,12684,846,polygon,False,True,America/New_York,3,28036 28035,1840016353
2,Huntersville,Huntersville,NC,North Carolina,37119,Mecklenburg,37119,Mecklenburg,35.4057,-80.8729,56212,535,polygon,False,True,America/New_York,3,28078 28070,1840016349
3,Cornelius,Cornelius,NC,North Carolina,37119,Mecklenburg,37119,Mecklenburg,35.4731,-80.8822,29191,951,polygon,False,True,America/New_York,3,28031,1840016352
4,Matthews,Matthews,NC,North Carolina,37119,Mecklenburg,37119,Mecklenburg,35.1195,-80.7101,32117,724,polygon,False,True,America/New_York,3,28105 28106,1840016350
5,Charlotte,Charlotte,NC,North Carolina,37119,Mecklenburg,37119,Mecklenburg,35.2079,-80.8304,1467362,1081,polygon,False,True,America/New_York,1,28269 28262 28105 28280 28282 28209 28208 2820...,1840014557
6,Mint Hill,Mint Hill,NC,North Carolina,37119,Mecklenburg,37119|37179,Mecklenburg|Union,35.1781,-80.6532,26748,426,polygon,False,True,America/New_York,3,28227,1840016351


In this section, we view the location of the various cities.
Huntersville, Cornelius and Davidson are in the north of Charlotte while others are in the south.

In [20]:
charlotte_map = folium.Map(location = [CLT_LAT, CLT_LON], zoom_start = 6)

for city, lat, lng in zip(Charlotte_NC['city'], 
                          Charlotte_NC['lat'], 
                          Charlotte_NC['lng']):
    label = '{}'.format(city)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
        [lat, lng],
        radius = 4,
        popup = label,
        color = 'green',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.5,
        parse_html = False).add_to(charlotte_map)  
    
charlotte_map

In this section, we list the basic attributes of Charlotte, NC.

In [23]:
print("Analyze the following City: {}".format(Charlotte_NC.loc[5, 'city']))

Analyze the following City: Charlotte


In [24]:
Explore = Charlotte_NC.loc[5, 'city']
Explore_Lat = Charlotte_NC.loc[5, 'lat']
Explore_Long = Charlotte_NC.loc[5, 'lng']
print('The city of {} is locationed at {} Latitude and {} Longitude '.format(Explore, 
                                                               Explore_Lat, 
                                                               Explore_Long))

The city of Charlotte is locationed at 35.2079 Latitude and -80.8304 Longitude 


In this section, we secure the locations from FourSqaure for the cities near Charlotte, NC.

In [25]:
url = ('https://api.foursquare.com/v2/venues/explore?client_id={}'
       '&client_secret={}&v={}&ll={},{}&radius={}&limit={}').format(CLIENT_ID, 
                                                                  CLIENT_SECRET, 
                                                                  VERSION, 
                                                                  Explore_Lat, 
                                                                  Explore_Long, 
                                                                  RADIUS,
                                                                  LIMIT)
Top100 = requests.get(url).json()
Top100

{'meta': {'code': 200, 'requestId': '5e84f0fe963d29001bf1548e'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'},
    {'name': '$-$$$$', 'key': 'price'}]},
  'headerLocation': 'Charlotte',
  'headerFullLocation': 'Charlotte',
  'headerLocationGranularity': 'city',
  'totalResults': 238,
  'suggestedBounds': {'ne': {'lat': 35.25290004500005,
    'lng': -80.77542762675448},
   'sw': {'lat': 35.16289995499996, 'lng': -80.88537237324552}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b0b230bf964a5208c2d23e3',
       'name': "Trader Joe's",
       'location': {'address': '1133 Metropolitan Ave',
        'crossStreet': 'Charlottetowne Ave',
        'lat': 35.21160846824355,
        'lng': -80.83546836626202

In this section, we setup placeholders to capture category and venues.

In [26]:
   def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In this section, we capture topp 100 venue results.

In [27]:
venue_results = Top100['response']['groups'][0]['items']
top_venues = json_normalize(venue_results)

# Filter the columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
top_venues = top_venues.loc[:, filtered_columns]

# Filter the category for each row
top_venues['venue.categories'] = top_venues.apply(get_category_type, axis = 1)

# Clean all column names
top_venues.columns = [col.split(".")[-1] for col in top_venues.columns]

top_venues.head(10)

,name,categories,lat,lng
0,Trader Joe's,Grocery Store,35.211608,-80.835468
1,Mama Ricotta's,Italian Restaurant,35.210081,-80.836012
2,Viva Chicken Elizabeth Avenue,Peruvian Restaurant,35.213407,-80.825966
3,Little Sugar Creek Greenway,Park,35.210849,-80.835981
4,Earl's Grocery,Deli / Bodega,35.213588,-80.826343
5,Duke Mansion,Bed & Breakfast,35.201550,-80.827493
6,Customshop,New American Restaurant,35.213828,-80.826544
7,Tropical Smoothie Cafe,Café,35.209687,-80.835677
8,Laurel Market,Wine Bar,35.202201,-80.824301
9,Sabor Latin Street Grill,Latin American Restaurant,35.216258,-80.821734


In [29]:
print('{} venues were returned by Foursquare.'.format(top_venues.shape[0]))

100 venues were returned by Foursquare.


# Explore the city of Charlotte, NC

In [30]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City', 
                  'City Latitude', 
                  'City Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In this section, we list the cities in the Charlotte Metropolitan Area ( county = Mecklenburg )

In [31]:
charlotte_venues = getNearbyVenues(names=Charlotte_NC['city'],
                                   latitudes=Charlotte_NC['lat'],
                                   longitudes=Charlotte_NC['lng']
                                  )

Pineville
Davidson
Huntersville
Cornelius
Matthews
Charlotte
Mint Hill


In [32]:
# Check Dataframe Size
print(charlotte_venues.shape)
charlotte_venues

(57, 7)


,City,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Pineville,35.0864,-80.8915,Pintville Craft Beer,35.085629,-80.891111,Beer Garden
1,Pineville,35.0864,-80.8915,Kit's Trackside Crafts,35.085960,-80.891085,Beer Bar
2,Pineville,35.0864,-80.8915,OooWee BBQ,35.084593,-80.890404,Food Truck
3,Pineville,35.0864,-80.8915,Zafran Kabab Palace,35.087350,-80.886476,Indian Restaurant
4,Pineville,35.0864,-80.8915,The Dive-N Diner,35.085959,-80.887051,Diner
5,Pineville,35.0864,-80.8915,Rite Aid,35.085890,-80.886333,Pharmacy
6,Pineville,35.0864,-80.8915,unwind tea and coffee,35.085382,-80.891431,Tea Room
7,Pineville,35.0864,-80.8915,Yarn Shop By Rainy Day Creations,35.085351,-80.890695,Hobby Shop
8,Pineville,35.0864,-80.8915,Um’s Corner Cafe,35.085509,-80.891084,Café
9,Pineville,35.0864,-80.8915,Shell,35.084880,-80.886362,Gas Station


In this section, we list venues by city.

In [33]:
charlotte_venues.groupby('City').count()

,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
City,,,,,,
Charlotte,15,15,15,15,15,15
Cornelius,1,1,1,1,1,1
Davidson,2,2,2,2,2,2
Huntersville,5,5,5,5,5,5
Matthews,12,12,12,12,12,12
Mint Hill,9,9,9,9,9,9
Pineville,13,13,13,13,13,13


In [34]:
# Let's find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(charlotte_venues['Venue Category'].unique())))

There are 46 uniques categories.


# Analyze Each City

In [35]:
# one hot encoding
charlotte_onehot = pd.get_dummies(charlotte_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
charlotte_onehot['City'] = charlotte_venues['City'] 

# move neighborhood column to the first column
fixed_columns = [charlotte_onehot.columns[-1]] + list(charlotte_onehot.columns[:-1])
charlotte_onehot = charlotte_onehot[fixed_columns]

charlotte_onehot.head()

,City,American Restaurant,Arts & Crafts Store,Bakery,Bar,Baseball Field,Beer Bar,Beer Garden,Big Box Store,Brewery,Café,Camera Store,Construction & Landscaping,Cosmetics Shop,Deli / Bodega,Diner,Donut Shop,Dry Cleaner,Fondue Restaurant,Food Truck,Furniture / Home Store,Gas Station,Gym,Hobby Shop,Home Service,Ice Cream Shop,Indian Restaurant,Kids Store,Martial Arts Dojo,Men's Store,Mobile Phone Shop,Optical Shop,Pharmacy,Pizza Place,Pool,Pool Hall,Record Shop,Residential Building (Apartment / Condo),Restaurant,Sandwich Place,Shipping Store,Spa,Tea Room,Theater,Theme Park Ride / Attraction,Wine Shop,Women's Store
0,Pineville,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Pineville,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Pineville,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Pineville,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Pineville,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [36]:
# Charlotte OneHot Dataframe Shape
charlotte_onehot.shape

(57, 47)

In [37]:
#Group Cities
charlotte_grouped = charlotte_onehot.groupby('City').mean().reset_index()
charlotte_grouped

,City,American Restaurant,Arts & Crafts Store,Bakery,Bar,Baseball Field,Beer Bar,Beer Garden,Big Box Store,Brewery,Café,Camera Store,Construction & Landscaping,Cosmetics Shop,Deli / Bodega,Diner,Donut Shop,Dry Cleaner,Fondue Restaurant,Food Truck,Furniture / Home Store,Gas Station,Gym,Hobby Shop,Home Service,Ice Cream Shop,Indian Restaurant,Kids Store,Martial Arts Dojo,Men's Store,Mobile Phone Shop,Optical Shop,Pharmacy,Pizza Place,Pool,Pool Hall,Record Shop,Residential Building (Apartment / Condo),Restaurant,Sandwich Place,Shipping Store,Spa,Tea Room,Theater,Theme Park Ride / Attraction,Wine Shop,Women's Store
0,Charlotte,0.000000,0.000000,0.133333,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.066667,0.0,0.000000,0.000000,0.000000,0.066667,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.066667,0.0,0.000000,0.066667,0.066667,0.0,0.000000,0.066667,0.0,0.000000,0.066667,0.066667,0.000000,0.000000,0.000000,0.066667,0.000000,0.066667,0.000000,0.0,0.066667
1,Cornelius,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000
2,Davidson,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.5,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
3,Huntersville,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.2,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.2,0.000000,0.000000,0.000000,0.2,0.000000,0.000000,0.2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
4,Matthews,0.083333,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.083333,0.0,0.000000,0.000000,0.0,0.083333,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.166667,0.000000,0.083333,0.000000,0.083333,0.000000,0.000000,0.0,0.083333,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.083333,0.0,0.000000
5,Mint Hill,0.000000,0.111111,0.000000,0.111111,0.000000,0.111111,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.222222,0.0,0.000000,0.000000,0.000000,0.111111,0.111111,0.111111,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
6,Pineville,0.000000,0.000000,0.000000,0.000000,0.000000,0.076923,0.076923,0.000000,0.0,0.076923,0.000000,0.0,0.000000,0.076923,0.076923,0.000000,0.000000,0.000000,0.076923,0.076923,0.076923,0.000000,0.076923,0.000000,0.000000,0.076923,0.0,0.000000,0.000000,0.000000,0.0,0.076923,0.000000,0.0,0.076923,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.076923,0.000000,0.000000,0.0,0.000000


In [38]:
#Confirm New Shape
charlotte_grouped.shape

(7, 47)

In [39]:
# Let's print each city along with the top 5 most common venues

num_top_venues = 5

for hood in charlotte_grouped['City']:
    print("----"+hood+"----")
    temp = charlotte_grouped[charlotte_grouped['City'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Charlotte----
            venue  freq
0          Bakery  0.13
1   Women's Store  0.07
2    Camera Store  0.07
3  Ice Cream Shop  0.07
4     Men's Store  0.07


----Cornelius----
                 venue  freq
0            Wine Shop   1.0
1  American Restaurant   0.0
2            Pool Hall   0.0
3    Indian Restaurant   0.0
4           Kids Store   0.0


----Davidson----
                        venue  freq
0              Cosmetics Shop   0.5
1  Construction & Landscaping   0.5
2                 Record Shop   0.0
3                  Kids Store   0.0
4           Martial Arts Dojo   0.0


----Huntersville----
                 venue  freq
0  American Restaurant   0.2
1           Kids Store   0.2
2         Optical Shop   0.2
3              Brewery   0.2
4                 Pool   0.2


----Matthews----
                          venue  freq
0        Furniture / Home Store  0.17
1           American Restaurant  0.08
2                Cosmetics Shop  0.08
3  Theme Park Ride / Attraction  0.08
4  

In [40]:
# Function to sort the venues in descending order.

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [41]:
# New dataframe and display the top 10 venues for each neighborhood.

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['City']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['City'] = charlotte_grouped['City']

for ind in np.arange(charlotte_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(charlotte_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Charlotte,Bakery,Women's Store,Record Shop,Donut Shop,Ice Cream Shop,Indian Restaurant,Camera Store,Men's Store,Mobile Phone Shop,Fondue Restaurant
1,Cornelius,Wine Shop,Women's Store,Construction & Landscaping,Gas Station,Furniture / Home Store,Food Truck,Fondue Restaurant,Dry Cleaner,Donut Shop,Diner
2,Davidson,Construction & Landscaping,Cosmetics Shop,Women's Store,Gas Station,Furniture / Home Store,Food Truck,Fondue Restaurant,Dry Cleaner,Donut Shop,Diner
3,Huntersville,American Restaurant,Brewery,Kids Store,Optical Shop,Pool,Baseball Field,Bakery,Food Truck,Fondue Restaurant,Dry Cleaner
4,Matthews,Furniture / Home Store,American Restaurant,Big Box Store,Home Service,Dry Cleaner,Martial Arts Dojo,Cosmetics Shop,Gym,Baseball Field,Theme Park Ride / Attraction


# Cluster Charlotte, North Carolina Cities

In [42]:
# Run K Means

# set number of clusters
kclusters = 3

charlotte_grouped_clustering = charlotte_grouped.drop('City', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(charlotte_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 2, 1, 0, 0, 0, 0], dtype=int32)

In [47]:
##### add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

charlotte_merged = Charlotte_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
charlotte_merged = charlotte_merged.join(neighborhoods_venues_sorted.set_index('City'), on='city')

charlotte_merged.head(100) # check the last columns!

,city,city_ascii,state_id,state_name,county_fips,county_name,county_fips_all,county_name_all,lat,lng,population,density,source,military,incorporated,timezone,ranking,zips,id,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Trenton,Trenton,NC,North Carolina,37103,Jones,37103,Jones,35.0640,-77.3553,285,484,polygon,False,True,America/New_York,3,28585,1840016445,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Stanfield,Stanfield,NC,North Carolina,37167,Stanly,37167,Stanly,35.2336,-80.4303,1517,131,polygon,False,True,America/New_York,3,28163,1840016365,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Sanford,Sanford,NC,North Carolina,37105,Lee,37105,Lee,35.4874,-79.1772,34557,389,polygon,False,True,America/New_York,3,27330 27332 27331,1840015421,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Cofield,Cofield,NC,North Carolina,37091,Hertford,37091,Hertford,36.3567,-76.9105,382,46,polygon,False,True,America/New_York,3,27922,1840017704,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Thomasville,Thomasville,NC,North Carolina,37057,Davidson,37057,Davidson,35.8813,-80.0807,26615,612,polygon,False,True,America/New_York,3,27360 27361,1840015361,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Aurora,Aurora,NC,North Carolina,37013,Beaufort,37013,Beaufort,35.3030,-76.7889,512,212,polygon,False,True,America/New_York,3,27806,1840015398,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Casar,Casar,NC,North Carolina,37045,Cleveland,37045,Cleveland,35.5127,-81.6168,293,64,polygon,False,True,America/New_York,3,28020,1840016333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Mebane,Mebane,NC,North Carolina,37001,Alamance,37001|37135,Alamance|Orange,36.0876,-79.2756,14973,633,polygon,False,True,America/New_York,3,27302,1840015328,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Vass,Vass,NC,North Carolina,37125,Moore,37125,Moore,35.2533,-79.2845,774,93,polygon,False,True,America/New_York,3,28394,1840016357,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Forest Hills,Forest Hills,NC,North Carolina,37099,Jackson,37099,Jackson,35.2958,-83.1954,382,297,polygon,False,True,America/New_York,3,28723,1840001769,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
charlotte_merged.head()

,city,city_ascii,state_id,state_name,county_fips,county_name,county_fips_all,county_name_all,lat,lng,population,density,source,military,incorporated,timezone,ranking,zips,id,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Trenton,Trenton,NC,North Carolina,37103,Jones,37103,Jones,35.0640,-77.3553,285,484,polygon,False,True,America/New_York,3,28585,1840016445,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Stanfield,Stanfield,NC,North Carolina,37167,Stanly,37167,Stanly,35.2336,-80.4303,1517,131,polygon,False,True,America/New_York,3,28163,1840016365,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Sanford,Sanford,NC,North Carolina,37105,Lee,37105,Lee,35.4874,-79.1772,34557,389,polygon,False,True,America/New_York,3,27330 27332 27331,1840015421,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Cofield,Cofield,NC,North Carolina,37091,Hertford,37091,Hertford,36.3567,-76.9105,382,46,polygon,False,True,America/New_York,3,27922,1840017704,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Thomasville,Thomasville,NC,North Carolina,37057,Davidson,37057,Davidson,35.8813,-80.0807,26615,612,polygon,False,True,America/New_York,3,27360 27361,1840015361,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
# create map
map_clusters = folium.Map(location=[lat, lng], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lng, poi, cluster in zip(charlotte_merged['lat'], charlotte_merged['lng'], charlotte_merged['city'], charlotte_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        #color=rainbow[cluster-1],
        fill=True,
        #fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Cluster #1 Analysis:
In this section, we study the attributes of the first cluster. Cluster #1 consisted of 5 cities that had a wide variety of location venues and had large population in them.

In [53]:
#Examine Cluster 1
charlotte_merged.loc[charlotte_merged['Cluster Labels'] == 0, charlotte_merged.columns[[1] + list(range(5, charlotte_merged.shape[1]))]]

,city_ascii,county_name,county_fips_all,county_name_all,lat,lng,population,density,source,military,incorporated,timezone,ranking,zips,id,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,Pineville,Mecklenburg,37119,Mecklenburg,35.0864,-80.8915,8746,508,polygon,False,True,America/New_York,3,28134,1840016354,0.0,Hobby Shop,Pool Hall,Food Truck,Indian Restaurant,Diner,Deli / Bodega,Pharmacy,Gas Station,Café,Beer Garden
92,Huntersville,Mecklenburg,37119,Mecklenburg,35.4057,-80.8729,56212,535,polygon,False,True,America/New_York,3,28078 28070,1840016349,0.0,American Restaurant,Brewery,Kids Store,Optical Shop,Pool,Baseball Field,Bakery,Food Truck,Fondue Restaurant,Dry Cleaner
437,Matthews,Mecklenburg,37119,Mecklenburg,35.1195,-80.7101,32117,724,polygon,False,True,America/New_York,3,28105 28106,1840016350,0.0,Furniture / Home Store,American Restaurant,Big Box Store,Home Service,Dry Cleaner,Martial Arts Dojo,Cosmetics Shop,Gym,Baseball Field,Theme Park Ride / Attraction
475,Charlotte,Mecklenburg,37119,Mecklenburg,35.2079,-80.8304,1467362,1081,polygon,False,True,America/New_York,1,28269 28262 28105 28280 28282 28209 28208 2820...,1840014557,0.0,Bakery,Women's Store,Record Shop,Donut Shop,Ice Cream Shop,Indian Restaurant,Camera Store,Men's Store,Mobile Phone Shop,Fondue Restaurant
632,Mint Hill,Mecklenburg,37119|37179,Mecklenburg|Union,35.1781,-80.6532,26748,426,polygon,False,True,America/New_York,3,28227,1840016351,0.0,Pizza Place,Gym,Arts & Crafts Store,Bar,Shipping Store,Sandwich Place,Restaurant,Beer Bar,Construction & Landscaping,Food Truck


# Cluster #2 Analysis:
In this section, we study the attributes of second cluster. Cluster #2 was the city of Davidson itself which was more blue collar based on the venues in it.

In [54]:
#Examine Cluster 2
charlotte_merged.loc[charlotte_merged['Cluster Labels'] == 1, charlotte_merged.columns[[1] + list(range(5, charlotte_merged.shape[1]))]]

,city_ascii,county_name,county_fips_all,county_name_all,lat,lng,population,density,source,military,incorporated,timezone,ranking,zips,id,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
72,Davidson,Mecklenburg,37119,Mecklenburg,35.4862,-80.8271,12684,846,polygon,False,True,America/New_York,3,28036 28035,1840016353,1.0,Construction & Landscaping,Cosmetics Shop,Women's Store,Gas Station,Furniture / Home Store,Food Truck,Fondue Restaurant,Dry Cleaner,Donut Shop,Diner


# Cluster #3 Analysis:
In this section, we study the attributes of third cluster. 
Cluster #3 was the city of Cornelius, which seems more up and coming with larger population base and higher density.

In [55]:
#Examine Cluster 3
charlotte_merged.loc[charlotte_merged['Cluster Labels'] == 2, charlotte_merged.columns[[1] + list(range(5, charlotte_merged.shape[1]))]]

,city_ascii,county_name,county_fips_all,county_name_all,lat,lng,population,density,source,military,incorporated,timezone,ranking,zips,id,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
225,Cornelius,Mecklenburg,37119,Mecklenburg,35.4731,-80.8822,29191,951,polygon,False,True,America/New_York,3,28031,1840016352,2.0,Wine Shop,Women's Store,Construction & Landscaping,Gas Station,Furniture / Home Store,Food Truck,Fondue Restaurant,Dry Cleaner,Donut Shop,Diner


# 4.	Results and Discussion

Based on the above observations the following 5 suburban areas in the Charlotte Metropolitan Ares are good for people looking to buy a house to get a more mature infrastructure and entertainment locations:

1.	Pineville
2.	Huntersville
3.	Matthews
4.	Charlotte
5.	Mint Hill

From a business perspective, the following two cities offer good business potential.

1.	Cornelius
2.	Davidson

Of the two above, Cornelius has a larger population base and higher density so the potential for business to succeed will be higher in Cornelius.

# 5.	Conclusion

Charlotte seems to blossoming into a new Mega Metropolis. However, not every area is experiencing the same level of growth. The town of Cornelius seems to be on the cusp of breaking into a good growth momentum and may be a good area for first time home buyers but definitely a good place top open a business. The city of Davidson also seems like a good business potential opportunity given the low number of venues there. Since these two cities are next to each other, there is a good potential for Davidson to benefit from the growth in Cornelius and possible upside from trickle down economic effect to sway its fortunes in the near future. 

However for now the following cities are good places for home buyers due to the abundance of population and services:
1.	Pineville
2.	Huntersville
3.	Matthews
4.	Charlotte
5.	Mint Hill
